Realizo la prueba del pipeline de datos con el vectorizador `TexVectorizer`, entrenando distintintos clasificadores
para seleccionar el de mejor desempeño

In [1]:
from modules.create_csv import crear_csv
from modules.text_vectorizer import TextVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\je_su\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\je_su\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
datos = crear_csv("./data/frases.json")
datos.head()

reclamo             etiqueta
0           Falta agua en el dispenser del módulo 1.           maestranza
1            la compu 23 del laboratorio 3 se tilda.  soporte informático
2  La ventana del Departamento informática quedó ...           maestranza
3   la cerradura del laboratorio de física se rompió   secretaría técnica
4        la pileta del baño del módulo 1 pierde agua   secretaría técnica

In [3]:
X = datos['reclamo']
y = datos['etiqueta'] #label_encoder.fit_transform(datos['etiqueta'])

In [4]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)

In [10]:
vectorizer = TextVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_train_vec[15]

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [22]:
grid = [
    {
        'classifier': [SVC()],
        'classifier__C': [0.01, 0.1, 0.5, 1, 5, 10, 15, 100], 
        'classifier__kernel': ['rbf', 'sigmoid', 'linear', 'poly'], 
        'classifier__gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1],
        'classifier__degree': [2, 3]
    },
    # {
    #     'classifier': [MLPClassifier()],
    #     'classifier__hidden_layer_sizes': [(5,), (10,), (5, 5), (10, 5), (10, 10)],
    #     'classifier__activation': ['tanh', 'relu'],
    #     'classifier__solver': ['adam'],
    #     'classifier__alpha': [0.0001, 0.001, 0.01],
    #     'classifier__learning_rate': ['constant', 'adaptive'],
    #     'classifier__max_iter': [200, 400, 600]
    # },
    {
        'classifier': [RandomForestClassifier(random_state=42)],
        'classifier__random_state': [0,10, 100, 42],
        'classifier__n_estimators': [10, 50, 100, 200],
        'classifier__max_features': ['sqrt', 'log2'],
        'classifier__max_depth': [None, 10, 20, 30],
        'classifier__min_samples_split': [2, 5, 10],
        'classifier__min_samples_leaf': [1, 2, 4]
    }
]

pipe = Pipeline([
    ("vectorizer", TextVectorizer()),
    ("scaler", StandardScaler()),
    ("classifier", SVC())
])

grid_search = GridSearchCV(pipe, grid, cv=3, n_jobs=-1, verbose=1)

In [23]:
best_clf = grid_search.fit(X_train, y_train)
best_clf

Fitting 3 folds for each of 1536 candidates, totalling 4608 fits


d:\REPOSITORIOS\pa-tps-codigos-iniciales-para-estudiantes\claims_classifier\.venv\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('vectorizer', TextVectorizer()),
                                       ('scaler', StandardScaler()),
                                       ('classifier', SVC())]),
             n_jobs=-1,
             param_grid=[{'classifier': [SVC()],
                          'classifier__C': [0.01, 0.1, 0.5, 1, 5, 10, 15, 100],
                          'classifier__degree': [2, 3],
                          'classifier__gamma': [0.001, 0.01, 0.05, 0.1, 0.5, 1],
                          'classifier__kernel': ['rbf', 'sigmoid', 'linear',
                                                 'poly']},
                         {'classifier': [RandomForestClassifier(random_state=42)],
                          'classifier__max_depth': [None, 10, 20, 30],
                          'classifier__max_features': ['sqrt', 'log2'],
                          'classifier__min_samples_leaf': [1, 2, 4],
                          'classifier__min_samples_split': [2, 5, 10],
                          'classifier__n_estimators': [10, 50, 100, 200],
                          'classifier__random_state': [0, 10, 100, 42]}],
             verbose=1)

In [24]:
best_clf.score(X_test, y_test)

0.8421052631578947